<a href="https://colab.research.google.com/github/thravt/FederatedAggregation/blob/main/CNNAvg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
def average_weights(models):
    weights = [model.get_weights() for model in models]
    new_weights = list()
    for weights_list_tuple in zip(*weights):
        new_weights.append(np.array(weights_list_tuple).mean(axis=0))
    return new_weights

In [ ]:
def makecnn():
  model = tf.keras.models.Sequential()
  model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.Flatten())
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(10))
  return model

In [ ]:
import matplotlib.pyplot as plt
import PIL
import pandas as pd

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


For this I will use 5 local models, and a central model.

In [ ]:
modelC = makecnn()
modellist = []
for i in range(5):
  modellist.append(makecnn())
  modellist[i].set_weights(list(modelC.get_weights()))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
#Getting splits:
imagelist = [[], [], [], [], []]
labellist = [[], [], [], [], []]
for i in range (len(train_labels)):
  imagelist[i % 5].append(train_images[i])
  labellist[i % 5].append(train_labels[i])


In [ ]:
for i in range(5):
  imagelist[i] = np.stack(imagelist[i])
  labellist[i] = np.stack(labellist[i])

In [ ]:
for i in range(5):
  modellist[i].compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', keras.metrics.F1Score])

In [ ]:
modelC.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', keras.metrics.F1Score])

I know that we planned on variance, drift, and convergence time, but those don't have metrics in base tensorflow, so those will have to be computed manually.

In [ ]:
for i in range(5):
  for j in range(5):
    modellist[j].set_weights(list(modelC.get_weights()))
  for j in range(5):
    modellist[j].fit(imagelist[j], labellist[j], epochs = 5)
  averageweights = average_weights(modellist)
  modelC.set_weights(averageweights)
  print(modelC.evaluate(test_images, test_labels))

Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 19s 50ms/step - accuracy: 0.1922 - f1_score: 0.1949 - loss: 4.2946
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 18s 42ms/step - accuracy: 0.3674 - f1_score: 0.1954 - loss: 1.7280
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 42ms/step - accuracy: 0.4348 - f1_score: 0.1955 - loss: 1.5318
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 42ms/step - accuracy: 0.5076 - f1_score: 0.1955 - loss: 1.3654
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 21s 43ms/step - accuracy: 0.5440 - f1_score: 0.1955 - loss: 1.2817
Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 15s 42ms/step - accuracy: 0.1872 - f1_score: 0.1951 - loss: 4.2154
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 42ms/step - accuracy: 0.3683 - f1_score: 0.1954 - loss: 1.7395
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 13s 42ms/step - accuracy: 0.4488 - f1_score: 0.1955 - loss: 1.5409
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 13s 42ms/step - accuracy: 0.4902 - f1_score: 0.1955 - loss: 1.4165
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 21s 42m

In [ ]:
modelC.evaluate(test_images, test_labels)

313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.6444 - f1_score: 0.1956 - loss: 1.2690


[1.264356017112732,
 0.6434999704360962,
 <tf.Tensor: shape=(10,), dtype=float32, numpy=
 array([0.06036585, 0.12125681, 0.11064368, 0.160425  , 0.16723616,
        0.248466  , 0.22514196, 0.27041474, 0.28249425, 0.30952328],
       dtype=float32)>]